# Exploring the __GAMA__ spectroscopic catalog
notebook by _Alex Malz (GCCL@RUB)_, (add your name here)

In [ ]:
from astropy.io import fits
import corner
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

We want to get the spectra of galaxies matching conditions found [here](http://www.gama-survey.org/dr3/schema/table.php?id=31), specifically sorting them by redshift and redshift quality as well as field, since we probably only want to get environment statistics for subsets by location.

In [ ]:
with fits.open('SpecObj.fits') as hdul:
    hdul.info()
    print(hdul[1].header)
    df = pd.DataFrame(np.array(hdul[1].data).byteswap().newbyteorder())

In [ ]:
plt.hist(df['Z'], bins=[0., 0.042, 0.080, 0.130, 0.221, 0.317, 0.418, 0.525, 0.640, 0.764, 0.897, 1.041, 1.199, 1.372, 1.562, 1.772, 2.007, 2.269, 2.565, 2.899, 3.])
plt.semilogy()
plt.xlabel('redshift')

In [ ]:
corner.corner(np.array([df['RA'], df['DEC']]).T, labels=['RA', 'DEC'], show_titles=True)

In [ ]:
for i in range(5):
    quality = df.loc[df['NQ'] == i+1, 'Z']
    plt.hist(quality, alpha=0.5, label=str(i+1))
plt.legend(loc='upper right')
plt.semilogy()
plt.xlim(0., 3.5)
plt.xlabel('Z')
plt.title('redshift distributions by quality flag "NQ"')

## select and download spectra based on redshift and angular coordinates

In [ ]:
RA_bin_ends = [0., 80., 160., 200., 360.]
subsamples = []
for i in range(len(RA_bin_ends)-1):
    subsamples.append(df.loc[(df['RA'] >= RA_bin_ends[i]) & (df['RA'] < RA_bin_ends[i+1]), ['RA', 'DEC', 'Z', 'NQ', 'URL']])
    print(len(subsamples[-1]))

# Next steps


## maybe multiply spectra by photometric filter transmission curves to get colors

## construct redshift-environment-SED/color relationship